In [1]:
import sys
import sklearn
import numpy as np
import os
import tarfile
import urllib
import pandas as pd  #provides a dataframe
import urllib.request



In [2]:
# For graph plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# mpl.rc('axes', labelsize=14)
# mpl.rc('xtick', labelsize=12)
# mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Get Data

In [3]:
#Loading the .csv files into panda dataframes



#Confirmed cases and Deaths from Johns Hopkins Uni
confirmed_cases_df = pd.read_csv('/Users/emmanueljumbo/dataScience/python/Dissertation/experimentation/data/Johns_Hopkins_Uni_COVID19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
deaths_df = pd.read_csv('/Users/emmanueljumbo/dataScience/python/Dissertation/experimentation/data/Johns_Hopkins_Uni_COVID19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
recovered_df = pd.read_csv('/Users/emmanueljumbo/dataScience/python/Dissertation/experimentation/data/Johns_Hopkins_Uni_COVID19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')


#Vaccination Data from the CDC
vaccination_df = pd.read_csv('/Users/emmanueljumbo/dataScience/python/Dissertation/experimentation/data/CDC_Vaccination_Data_US_County_Level.csv', dtype={'FIPS': 'str'})
# Manually specifing column type using "dtype=" cause the blow warning was recieved. 
# Plus made date=string just to I can load data; will fix that later.
# -------------- Warning Message ---
# /Users/emmanueljumbo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: 
# Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
# has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
# --------------
# solution idea gotten from:
#https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options


#Social Vulnerability Indices (SVI) BY CDC 
SVI_df = pd.read_csv('/Users/emmanueljumbo/dataScience/python/Dissertation/experimentation/data/SVI2018_US_COUNTY.csv')


#Rural-Urban Continuum Codes by U.S. Department Of Agriculture (USDA) 
RuralUrban_df = pd.read_csv('/Users/emmanueljumbo/dataScience/python/Dissertation/experimentation/data/ruralurbancodes2013.csv')


#Hospital Bed Utilization Data (State-LeveL) by U.S. Department of Health & Human Services (healthdata.gov) 
HospitalBed_df = pd.read_csv('/Users/emmanueljumbo/dataScience/python/Dissertation/experimentation/data/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries.csv')




# Make Copies of Above Dataframe
vaccination_US = vaccination_df
SVI_US = SVI_df
RuralUrban_US = RuralUrban_df
HospitalBed_US = HospitalBed_df


# Original  Data Examination & Preparation
1. UNPIVOT the data
2. Replace Name & Value (Columns & Rows)
3. Change Column Data Types
4. Drop Unnecessary Columns
5. Null Value Search
6. DataSet Examination
7. Sort Columns by Date
8. Cumulative Column Check
9. Create Cumulative Sum Column


####  UNPIVOT the data

In [4]:
#Confirmed cases and Deaths from Johns Hopkins Uni
confirmed_cases_US = pd.melt(confirmed_cases_df, id_vars=['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key'], var_name=['Date']) 
deaths_US = pd.melt(deaths_df, id_vars=['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population'], var_name=['Date']) 
recovered_global = pd.melt(recovered_df, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date']) 




#              How It Works:
# All columns in "id_vars" are not to be touched | unpivot all other columns into a New column named "Date"
# then combine all values of the old/ unpivoted columns(now "date") into a new column named "value" 
# RESULT: Data of the new datafame/ table can now be aggregated by value, rather than including hundreds of individual dates as columns in an aggregation.


####  Replace Name & Value (Columns & Rows)

In [5]:
# Changing Individual default Column OR Row Names


#Confirmed cases and Deaths from Johns Hopkins Uni
confirmed_cases_US.columns = confirmed_cases_US.columns.str.replace('value', 'Confirmed_Cases')
confirmed_cases_US = confirmed_cases_US.replace(to_replace ="Unassigned", value = np.NaN)
deaths_US.columns = deaths_US.columns.str.replace('value', 'Deaths')
deaths_US = deaths_US.replace(to_replace ="Unassigned", value = np.NaN)
recovered_global.columns = recovered_global.columns.str.replace('value', 'Recoveries')


# Vaccination Data from the CDC
vaccination_US = vaccination_US.replace(to_replace ="UNK", value = np.NaN)



#              What And Why:
#(UNK == null value) (Unassigned == null value) this is why I replaced all Unassigned, UNK in the dataset to blanks for easy identification and null handling. 
#  .replace IDEA FROM = https://www.geeksforgeeks.org/python-pandas-dataframe-replace/
#  np.NaN/pd.na IDEA FROM = https://stackoverflow.com/questions/34794067/how-to-set-a-cell-to-nan-in-a-pandas-dataframe


#### Change Column Data Types

In [6]:
# Converting column data types


#Confirmed cases and Deaths from Johns Hopkins Uni
confirmed_cases_US['Date'] = pd.to_datetime(confirmed_cases_US['Date'])
deaths_US['Date'] = pd.to_datetime(deaths_US['Date'])
recovered_global['Date'] = pd.to_datetime(recovered_global['Date'])

#Vaccination Data from the CDC
vaccination_US['Date'] = pd.to_datetime(vaccination_US['Date'])
vaccination_US['FIPS'] = pd.to_numeric(vaccination_US['FIPS'])

#Hospital Bed Utilization Data (State-LeveL) by U.S. Department of Health & Human Services (healthdata.gov) 
HospitalBed_US['date'] = pd.to_datetime(HospitalBed_US['date'])


#              How It Works:
# Simply converting all new columns to “date & time” data type
# Simply converting all vaccination_US's Date columns to “date & time” data type
# Simply converting all “FIPS” columns to “float” data type
# Simply converting all HospitalBed_US's date columns to “date & time” data type
# IDEA FROM = https://www.geeksforgeeks.org/how-to-convert-strings-to-floats-in-pandas-dataframe/


#### Drop Unnecessary Columns

In [7]:
#Drop Unnecessary Columns




#Confirmed cases and Deaths from Johns Hopkins Uni
confirmed_cases_US = confirmed_cases_US.drop(confirmed_cases_US.loc[:, "iso2":"code3"].columns, axis = 1)
confirmed_cases_US = confirmed_cases_US.drop("Country_Region", axis=1)
confirmed_cases_US = confirmed_cases_US.drop("Combined_Key", axis=1)
#----
deaths_US = deaths_US.drop(deaths_US.loc[:, "iso2":"code3"].columns, axis = 1)
deaths_US = deaths_US.drop("Country_Region", axis=1)
deaths_US = deaths_US.drop("Combined_Key", axis=1)


#Vaccination Data from the CDC
vaccination_US = vaccination_US.drop("MMWR_week", axis=1)
vaccination_US = vaccination_US.drop(vaccination_US.loc[:, "Series_Complete_12Plus":"Series_Complete_65PlusPop_Pct"].columns, axis = 1)
vaccination_US = vaccination_US.drop(vaccination_US.loc[:, "Administered_Dose1_Recip_12Plus":"Administered_Dose1_Recip_65PlusPop_Pct"].columns, axis = 1)
vaccination_US = vaccination_US.drop(vaccination_US.loc[:, "Series_Complete_Pop_Pct_SVI":"Series_Complete_65PlusPop_Pct_SVI"].columns, axis = 1)
vaccination_US = vaccination_US.drop(vaccination_US.loc[:, "Series_Complete_Pop_Pct_UR_Equity":"Series_Complete_65PlusPop_Pct_UR_Equity"].columns, axis = 1)
vaccination_US = vaccination_US.drop(vaccination_US.loc[:, "Series_Complete_5Plus":"Series_Complete_5PlusPop_Pct_UR_Equity"].columns, axis = 1)
vaccination_US = vaccination_US.drop(vaccination_US.loc[:, "Booster_Doses_18Plus":"Booster_Doses_65Plus"].columns, axis = 1)
vaccination_US = vaccination_US.drop(vaccination_US.loc[:, "Booster_Doses_18Plus_Vax_Pct":"Booster_Doses_65Plus_Vax_Pct"].columns, axis = 1)



#Social Vulnerability Indices (SVI) BY CDC 
SVI_US = SVI_US.drop("ST", axis=1)
# SVI_US = SVI_US.drop("ST_ABBR", axis=1)
# SVI_US = SVI_US.drop(SVI_US.loc[:, "LOCATION":"AREA_SQMI"].columns, axis = 1)
SVI_US = SVI_US.drop("LOCATION", axis=1)
SVI_US = SVI_US.drop("M_TOTPOP", axis=1)
SVI_US = SVI_US.drop("M_HU", axis=1)
SVI_US = SVI_US.drop("M_HH", axis=1)
SVI_US = SVI_US.drop("M_POV", axis=1)
SVI_US = SVI_US.drop("M_UNEMP", axis=1)
SVI_US = SVI_US.drop(SVI_US.loc[:, "M_PCI":"M_NOHSDP"].columns, axis = 1)
SVI_US = SVI_US.drop("M_AGE65", axis=1)
SVI_US = SVI_US.drop("M_AGE17", axis=1)
SVI_US = SVI_US.drop(SVI_US.loc[:, "E_DISABL":"M_SNGPNT"].columns, axis = 1)
SVI_US = SVI_US.drop(SVI_US.loc[:, "M_MINRTY":"M_GROUPQ"].columns, axis = 1)
SVI_US = SVI_US.drop("MP_POV", axis=1)
SVI_US = SVI_US.drop("MP_UNEMP", axis=1)
SVI_US = SVI_US.drop(SVI_US.loc[:, "MP_PCI":"MP_NOHSDP"].columns, axis = 1)
SVI_US = SVI_US.drop("MP_AGE65", axis=1)
SVI_US = SVI_US.drop("MP_AGE17", axis=1)
SVI_US = SVI_US.drop(SVI_US.loc[:, "EP_DISABL":"MP_SNGPNT"].columns, axis = 1)
SVI_US = SVI_US.drop(SVI_US.loc[:, "MP_MINRTY":"F_TOTAL"].columns, axis = 1)
SVI_US = SVI_US.drop("M_UNINSUR", axis=1)
SVI_US = SVI_US.drop(SVI_US.loc[:, "MP_UNINSUR":"E_DAYPOP"].columns, axis = 1)


# Just droping multiple columns from the dataset
# IDEA FROM= https://www.geeksforgeeks.org/how-to-drop-one-or-multiple-columns-in-pandas-dataframe/


#### NULL Value Search

In [8]:


print("Sum of NULL Values In Confirmed_cases_US: \n \n", confirmed_cases_US.isnull().sum())
print("confirmed_cases_US shape: ", confirmed_cases_US.shape)

print(" \n \n Sum of NULL Values In Deaths_US: \n \n", deaths_US.isnull().sum())
print("deaths_US shape: ", deaths_US.shape)

print(" \n \n Sum of NULL Values In vaccination_US: \n \n", vaccination_US.isnull().sum())
print("vaccination_US shape: ", vaccination_US.shape)


pd.set_option("display.max_rows", None)   # Lets you see ALL dataframe ROW.

print(" \n \n Sum of NULL Values In SVI_US: \n \n", SVI_US.isnull().sum())
print("SVI_US shape: ", SVI_US.shape)

print(" \n \n Sum of NULL Values In RuralUrban_US: \n \n", RuralUrban_US.isnull().sum())
print("RuralUrban_US shape: ", RuralUrban_US.shape)

print(" \n \n Sum of NULL Values In RuralUrban_US: \n \n", HospitalBed_US.isnull().sum())
print("HospitalBed_US shape: ", HospitalBed_US.shape)

Sum of NULL Values In Confirmed_cases_US: 
 
 UID                    0
FIPS                7140
Admin2             41412
Province_State         0
Lat                    0
Long_                  0
Date                   0
Confirmed_Cases        0
dtype: int64
confirmed_cases_US shape:  (2386188, 8)
 
 
 Sum of NULL Values In Deaths_US: 
 
 UID                   0
FIPS               7140
Admin2            41412
Province_State        0
Lat                   0
Long_                 0
Population            0
Date                  0
Deaths                0
dtype: int64
deaths_US shape:  (2386188, 9)
 
 
 Sum of NULL Values In vaccination_US: 
 
 Date                                           0
FIPS                                       22487
Recip_County                                   0
Recip_State                                  308
Series_Complete_Pop_Pct                        0
Series_Complete_Yes                            0
Completeness_pct                               0
Administe

#### DataSet Examination

In [9]:

## Primary DataSet Examination
# COVID19_US.tail()
# COVID19_US.head()
# print("HospitalBed_US shape: \n", HospitalBed_US.shape)
# print("\n \n HospitalBed_US head: \n", HospitalBed_US.tail())
# print("\n \n HospitalBed_US head: \n", HospitalBed_US.tail())
# print("\n \n HospitalBed_US info: \n", COVID19_US.info())
# print("\n \n HospitalBed_US Null Count: \n",HospitalBed_US.isnull().sum())
# display(HospitalBed_US.describe()) #Return stats on numerical attributes(all my Columns) in Data set


##  Secondary DataSet Examination
##  (For Examining specific DataSet Attributes)
# print(" ", deaths_US["FIPS"].dtypes)
# confirmed_cases_US["FIPS"].value_count
# display(HospitalBed_US["diagnosis"].value_counts()) #Returns the unique values of my Categorical attribuite
# print(" \n \n  \n \n Specific NULL Rows in Final Dataset:  \n \n", COVID19_US[COVID19_US['date'].isnull()].head(50))
# max(HospitalBed_US['date'])
# min(HospitalBed_US['date'])


##  Final DataSet Examination
# print(" \n \n  \n \n Specific NULL Rows in Final Dataset:  \n \n", COVID19_US[COVID19_US['date'].isnull()].tail(50))
# print("\n\n   \n", COVID19_US.shape)
# print("", COVID19_US[COVID19_US["Province_State"] == "Puerto Rico"].shape)



#### Sort Columns by Date

In [10]:
HospitalBed_US = HospitalBed_US.sort_values(by='date')

#Done cause the data (date column) was unorder.

#### Cumulative Column Check

In [11]:
# HospitalBed_US[HospitalBed_US["state"] == "TX"].head(50)


#                  STATUS
# vaccination_US  => CUMULATIVE
# SVI_US          => FIXED (values dont change over time)
# RuralUrban_US   => FIXED (values dont change over time)
# HospitalBed_US  => NOT CUMULATIVE
# confirmed_cases_df => CUMULATIVE
# deaths_df       => CUMULATIVE



#                     What And Why:
## Need to know if dataset are cumulative or not.
## DataSet is "cumulative" when values within its individual column continously increase(From earlier dates to more recent time)
## DataSet is "NOT cumulative" when values within its individual column are distributed at random.

#### Cumulative Column Creation

In [12]:

HospitalBed_US['inpatient_beds'] = HospitalBed_US.groupby(['state'])['inpatient_beds'].cumsum()
HospitalBed_US['inpatient_beds_used'] = HospitalBed_US.groupby(['state'])['inpatient_beds_used'].cumsum()
HospitalBed_US['inpatient_beds_used_covid'] = HospitalBed_US.groupby(['state'])['inpatient_beds_used_covid'].cumsum()
HospitalBed_US['critical_staffing_shortage_today_yes'] = HospitalBed_US.groupby(['state'])['critical_staffing_shortage_today_yes'].cumsum()
HospitalBed_US['inpatient_beds_utilization_numerator'] = HospitalBed_US.groupby(['state'])['inpatient_beds_utilization_numerator'].cumsum()
HospitalBed_US['inpatient_beds_utilization_denominator'] = HospitalBed_US.groupby(['state'])['inpatient_beds_utilization_denominator'].cumsum()
HospitalBed_US['percent_of_inpatients_with_covid_numerator'] = HospitalBed_US.groupby(['state'])['percent_of_inpatients_with_covid_numerator'].cumsum()
HospitalBed_US['percent_of_inpatients_with_covid_denominator'] = HospitalBed_US.groupby(['state'])['percent_of_inpatients_with_covid_denominator'].cumsum()
HospitalBed_US['inpatient_bed_covid_utilization_numerator'] = HospitalBed_US.groupby(['state'])['inpatient_bed_covid_utilization_numerator'].cumsum()
HospitalBed_US['inpatient_bed_covid_utilization_denominator'] = HospitalBed_US.groupby(['state'])['inpatient_bed_covid_utilization_denominator'].cumsum()
HospitalBed_US['staffed_adult_icu_bed_occupancy'] = HospitalBed_US.groupby(['state'])['staffed_adult_icu_bed_occupancy'].cumsum()
HospitalBed_US['staffed_icu_adult_patients_confirmed_and_suspected_covid'] = HospitalBed_US.groupby(['state'])['staffed_icu_adult_patients_confirmed_and_suspected_covid'].cumsum()
HospitalBed_US['staffed_icu_adult_patients_confirmed_covid'] = HospitalBed_US.groupby(['state'])['staffed_icu_adult_patients_confirmed_covid'].cumsum()
HospitalBed_US['total_adult_patients_hospitalized_confirmed_and_suspected_covid'] = HospitalBed_US.groupby(['state'])['total_adult_patients_hospitalized_confirmed_and_suspected_covid'].cumsum()
HospitalBed_US['adult_icu_bed_covid_utilization_numerator'] = HospitalBed_US.groupby(['state'])['adult_icu_bed_covid_utilization_numerator'].cumsum()
HospitalBed_US['adult_icu_bed_covid_utilization_denominator'] = HospitalBed_US.groupby(['state'])['adult_icu_bed_covid_utilization_denominator'].cumsum()
HospitalBed_US['adult_icu_bed_utilization_numerator'] = HospitalBed_US.groupby(['state'])['adult_icu_bed_utilization_numerator'].cumsum()
HospitalBed_US['adult_icu_bed_utilization_denominator'] = HospitalBed_US.groupby(['state'])['adult_icu_bed_utilization_denominator'].cumsum()



#                     What And Why:
# Making the above columns cumulative will aid in future calculations(eg. percentages), 
# visualisation and ML Modelling. Plus numeric columns of other imported time series
# datasets are cumulative.
# IDEA FROM = https://datascienceparichay.com/article/cumulative-sum-of-column-in-pandas-dataframe/

# Create New/ Final DataSet
1. Merge Original DataSet
2. View Merge Shapes
3. Copy & View Final DataSet
4. Search For Nulls in Final DataSet
5. Handle Null in Final DataSet
6. Rename Columns
7. Reindex Columns
8. Export CSV File

#### Merge Original DataSet

In [13]:
#Merging DataFrame Tables



# Confirmed cases and Deaths from Johns Hopkins Uni
#   Used "left join" cause, even if the a particular county doesnt have any cases, it's still gonna have its own rows in both tables
Cases_Deaths = pd.merge(confirmed_cases_US, deaths_US[['UID','Date','FIPS','Admin2','Province_State', 'Population', 'Deaths']], 
                                      how = 'left', 
                                      left_on = ['UID','Date','FIPS','Admin2','Province_State'], 
                                      right_on = ['UID','Date','FIPS','Admin2','Province_State'])


#Rural-Urban Continuum Codes by U.S. Department Of Agriculture (USDA)
Cases_Deaths_Metro = pd.merge(Cases_Deaths, RuralUrban_US[['FIPS', 'Metro_Status', 'State']], 
                                      how = 'left', 
                                      left_on = ['FIPS'], 
                                      right_on = ['FIPS'])


#Social Vulnerability Indices (SVI) BY CDC 
Case_Death_Metro_SVI = pd.merge(Cases_Deaths_Metro, SVI_US[['FIPS', 'E_TOTPOP', 'E_HU', 'E_HH', 'E_POV', 'E_UNEMP', 'E_PCI', 'E_AGE65', 'E_AGE17', 'E_MINRTY', 'EP_POV', 'EP_UNEMP', 'EP_PCI', 'EP_AGE65', 'EP_AGE17', 'EP_MINRTY', 'E_UNINSUR', 'EP_UNINSUR', 'AREA_SQMI']],
                                      how = 'left', 
                                      left_on = ['FIPS'], 
                                      right_on = ['FIPS'])

#Vaccination Data from the CDC
Case_Death_Metro_SVI_Vaxx = pd.merge(Case_Death_Metro_SVI, vaccination_US[['Date', 'FIPS', 'Recip_State', 'Recip_County','Series_Complete_Pop_Pct','Series_Complete_Yes','Completeness_pct','Administered_Dose1_Recip','Administered_Dose1_Pop_Pct','Administered_Dose1_Recip_5Plus','Administered_Dose1_Recip_5PlusPop_Pct','Booster_Doses','Booster_Doses_Vax_Pct']], 
                                      how = 'left', 
                                      left_on = ['Date', 'FIPS', 'State'], 
                                      right_on = ['Date', 'FIPS', 'Recip_State'])


#Hospital Bed Utilization Data (State-LeveL) by U.S. Department of Health & Human Services (healthdata.gov) 
Case_Death_Metro_SVI_Vaxx_Beds = pd.merge(Case_Death_Metro_SVI_Vaxx, HospitalBed_US[['state', 'date', 'inpatient_beds', 'inpatient_beds_used',
                                                                                    'inpatient_beds_used_covid', 'critical_staffing_shortage_today_yes', 
                                                                                    'inpatient_beds_utilization', 'inpatient_beds_utilization_numerator', 
                                                                                    'inpatient_beds_utilization_denominator', 'percent_of_inpatients_with_covid', 
                                                                                    'percent_of_inpatients_with_covid_numerator', 'percent_of_inpatients_with_covid_denominator', 
                                                                                    'inpatient_bed_covid_utilization', 'inpatient_bed_covid_utilization_numerator', 
                                                                                    'inpatient_bed_covid_utilization_denominator', 'staffed_adult_icu_bed_occupancy', 
                                                                                    'staffed_icu_adult_patients_confirmed_and_suspected_covid', 'staffed_icu_adult_patients_confirmed_covid', 
                                                                                    'total_adult_patients_hospitalized_confirmed_and_suspected_covid', 'adult_icu_bed_covid_utilization', 
                                                                                    'adult_icu_bed_covid_utilization_numerator', 'adult_icu_bed_covid_utilization_denominator', 
                                                                                    'adult_icu_bed_utilization', 'adult_icu_bed_utilization_numerator', 
                                                                                    'adult_icu_bed_utilization_denominator']], 
                                                                               how = 'left',
                                                                               left_on = ['State', 'Date'], 
                                                                               right_on = ['state', 'date'])                                    


#                     Merge
# IDEA FROM= https://www.geeksforgeeks.org/how-to-merge-two-csv-files-by-specific-column-using-pandas-in-python/


#                     Tried but NO Change (Number of nulls)
# Used “df["Country"].str.lower()” on SVI_US(STATE) and Cases_Deaths_Metro(Province_State) columns 
# to turn  both into all lowercase values in order to let both dataset merge on their individual state values 
# as well as the FIPS. This resulted in Zero changes to the number of null instances across the newly formed 
# dataset, so I took it out.  (NOTE: SVI_US(STATE) was all uppercase)
# IDEA FROM= https://stackoverflow.com/questions/29761915/case-insensitive-pandas-dataframe-merge


#### View Merge Shapes

In [14]:
#Print Shapes of each Merge Performed and Shapes of all DataSets Used




pd.set_option("display.max_columns", None)   # Lets you see ALL dataframe ROW.

print("**Shape Of DataSets Being Merged** ")
print("SVI_US shape:\n ", SVI_US.shape)
print("RuralUrban_US shape:\n ", RuralUrban_US.shape)
print("vaccination_US shape:\n ", vaccination_US.shape)
print("confirmed_cases_US shape:\n ", confirmed_cases_US.shape)
print("deaths_US shape:\n ", deaths_US.shape)
print("HospitalBed_US shape:\n ", HospitalBed_US.shape)

print("\n **Shape Of New DataSet After each Merged** ")
print("Cases_Deaths shape:\n ", Cases_Deaths.shape)
print("Cases_Deaths_Metro shape:\n ", Cases_Deaths_Metro.shape)
print("Case_Death_Metro_SVI shape:\n ", Case_Death_Metro_SVI.shape)
print("Case_Death_Metro_SVI_Vaxx shape:\n", Case_Death_Metro_SVI_Vaxx.shape)
print("Case_Death_Metro_SVI_Vaxx_Beds shape:\n ", Case_Death_Metro_SVI_Vaxx_Beds.shape)

# print("\n\n Case_Death_Metro_SVI_Vaxx head:\n\n ",Case_Death_Metro_SVI_Vaxx.head())
# print("\n\n Case_Death_Metro_SVI_Vaxx tail:\n\n ",Case_Death_Metro_SVI_Vaxx.tail())


**Shape Of DataSets Being Merged** 
SVI_US shape:
  (3142, 22)
RuralUrban_US shape:
  (3234, 7)
vaccination_US shape:
  (1279847, 15)
confirmed_cases_US shape:
  (2386188, 8)
deaths_US shape:
  (2386188, 9)
HospitalBed_US shape:
  (38135, 117)

 **Shape Of New DataSet After each Merged** 
Cases_Deaths shape:
  (2386188, 10)
Cases_Deaths_Metro shape:
  (2386188, 12)
Case_Death_Metro_SVI shape:
  (2386188, 30)
Case_Death_Metro_SVI_Vaxx shape:
 (2386188, 41)
Case_Death_Metro_SVI_Vaxx_Beds shape:
  (2386188, 66)


#### Copy & View Final DataSet

In [15]:
## Making A copy of last DataSet Merge
COVID19_US = Case_Death_Metro_SVI_Vaxx_Beds
COVID19_US.tail()

,UID,FIPS,Admin2,Province_State,Lat,Long_,Date,Confirmed_Cases,Population,Deaths,Metro_Status,State,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_AGE65,E_AGE17,E_MINRTY,EP_POV,EP_UNEMP,EP_PCI,EP_AGE65,EP_AGE17,EP_MINRTY,E_UNINSUR,EP_UNINSUR,AREA_SQMI,Recip_State,Recip_County,Series_Complete_Pop_Pct,Series_Complete_Yes,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,Booster_Doses,Booster_Doses_Vax_Pct,state,date,inpatient_beds,inpatient_beds_used,inpatient_beds_used_covid,critical_staffing_shortage_today_yes,inpatient_beds_utilization,inpatient_beds_utilization_numerator,inpatient_beds_utilization_denominator,percent_of_inpatients_with_covid,percent_of_inpatients_with_covid_numerator,percent_of_inpatients_with_covid_denominator,inpatient_bed_covid_utilization,inpatient_bed_covid_utilization_numerator,inpatient_bed_covid_utilization_denominator,staffed_adult_icu_bed_occupancy,staffed_icu_adult_patients_confirmed_and_suspected_covid,staffed_icu_adult_patients_confirmed_covid,total_adult_patients_hospitalized_confirmed_and_suspected_covid,adult_icu_bed_covid_utilization,adult_icu_bed_covid_utilization_numerator,adult_icu_bed_covid_utilization_denominator,adult_icu_bed_utilization,adult_icu_bed_utilization_numerator,adult_icu_bed_utilization_denominator
2386183,84056039,56039.0,Teton,Wyoming,43.935225,-110.589080,2022-01-04,6452,23464,14,Nonmetro,WY,23059.0,13680.0,9158.0,1619.0,210.0,53703.0,3135.0,4434.0,4246.0,7.1,1.4,53703.0,13.6,19.2,18.4,2305.0,10.0,3996.844622,WY,Teton County,88.7,20821.0,96.5,24074.0,95.0,24072.0,95.0,7825.0,37.6,WY,2022-01-04,1021682.0,458276.0,54517.0,2672.0,0.437422,455583.0,1002115.0,0.098529,54306.0,449435.0,0.042921,54268.0,982798.0,30632.0,12366.0,12022.0,51539.0,0.129771,12350.0,66300.0,0.437956,30631.0,70427.0
2386184,84056041,56041.0,Uinta,Wyoming,41.287818,-110.547578,2022-01-04,4191,20226,34,Nonmetro,WY,20609.0,8972.0,7735.0,2552.0,614.0,27009.0,2498.0,6071.0,2567.0,12.5,6.1,27009.0,12.1,29.5,12.5,2499.0,12.2,2081.719807,WY,Uinta County,46.3,9368.0,96.5,11156.0,55.2,11156.0,59.1,3140.0,33.5,WY,2022-01-04,1021682.0,458276.0,54517.0,2672.0,0.437422,455583.0,1002115.0,0.098529,54306.0,449435.0,0.042921,54268.0,982798.0,30632.0,12366.0,12022.0,51539.0,0.129771,12350.0,66300.0,0.437956,30631.0,70427.0
2386185,84090056,90056.0,NaN,Wyoming,0.000000,0.000000,2022-01-04,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2386186,84056043,56043.0,Washakie,Wyoming,43.904516,-107.680187,2022-01-04,1892,7805,39,Nonmetro,WY,8129.0,3868.0,3422.0,984.0,253.0,27556.0,1686.0,1942.0,1463.0,12.4,6.2,27556.0,20.7,23.9,18.0,1223.0,15.4,2238.672972,WY,Washakie County,40.3,3147.0,96.5,3460.0,44.3,3459.0,46.7,1355.0,43.1,WY,2022-01-04,1021682.0,458276.0,54517.0,2672.0,0.437422,455583.0,1002115.0,0.098529,54306.0,449435.0,0.042921,54268.0,982798.0,30632.0,12366.0,12022.0,51539.0,0.129771,12350.0,66300.0,0.437956,30631.0,70427.0
2386187,84056045,56045.0,Weston,Wyoming,43.839612,-104.567488,2022-01-04,1257,6927,16,Nonmetro,WY,7100.0,3565.0,3062.0,1171.0,117.0,29152.0,1340.0,1505.0,634.0,17.4,3.5,29152.0,18.9,21.2,8.9,898.0,13.3,2398.003891,WY,Weston County,34.7,2401.0,96.5,2693.0,38.9,2691.0,40.9,986.0,41.1,WY,2022-01-04,1021682.0,458276.0,54517.0,2672.0,0.437422,455583.0,1002115.0,0.098529,54306.0,449435.0,0.042921,54268.0,982798.0,30632.0,12366.0,12022.0,51539.0,0.129771,12350.0,66300.0,0.437956,30631.0,70427.0


#### Search For Nulls in Final DataSet

In [16]:
print(" \n \n Sum of NULL Values In COVID19_US: \n \n", COVID19_US.isnull().sum())

 
 
 Sum of NULL Values In COVID19_US: 
 
 UID                                                                      0
FIPS                                                                  7140
Admin2                                                               41412
Province_State                                                           0
Lat                                                                      0
Long_                                                                    0
Date                                                                     0
Confirmed_Cases                                                          0
Population                                                               0
Deaths                                                                   0
Metro_Status                                                         88536
State                                                                88536
E_TOTPOP                                                 

#### Handle Null in Final DataSet
1. Drop Rows
2. Drop Columns
3. Replace null values with '0'

In [17]:
##                              DROP Unwanted Rows
COVID19_US = COVID19_US.dropna(axis=0, subset=['FIPS']) 
COVID19_US = COVID19_US.dropna(axis=0, subset=['Admin2'])
COVID19_US = COVID19_US.dropna(axis=0, subset=['Metro_Status'])
COVID19_US = COVID19_US.dropna(axis=0, subset=['E_TOTPOP']) 



##                              DROP Unwanted Columns
#All columns removed were necessary for properly merging different datasets but would be irrelevant to ML model creation
COVID19_US = COVID19_US.drop("UID", axis=1)
COVID19_US = COVID19_US.drop("E_TOTPOP", axis=1)           # COVID19_US = COVID19_US.drop("Population", axis=1)
COVID19_US = COVID19_US.drop("Recip_State", axis=1)
COVID19_US = COVID19_US.drop("Recip_County", axis=1)
# COVID19_US = COVID19_US.drop("FIPS", axis=1)

## Remove Unnessary Hospital Bed Columns
#All columns removed were necessary for properly merging different datasets but would be irrelevant to ML model creation
COVID19_US = COVID19_US.drop("state", axis=1)
COVID19_US = COVID19_US.drop("date", axis=1)



##                               REPLACE null VAXX values with '0'
#These NULL values are as a result of the date in which Covid vaccines and busters were created 
#and distributed in relation to the date when the first covid case was recorded in the US
COVID19_US['Series_Complete_Pop_Pct'] = COVID19_US['Series_Complete_Pop_Pct'].fillna(-9999)
COVID19_US['Series_Complete_Yes'] = COVID19_US['Series_Complete_Yes'].fillna(-9999)
COVID19_US['Completeness_pct'] = COVID19_US['Completeness_pct'].fillna(-9999)
COVID19_US['Administered_Dose1_Recip'] = COVID19_US['Administered_Dose1_Recip'].fillna(-9999)
COVID19_US['Administered_Dose1_Pop_Pct'] = COVID19_US['Administered_Dose1_Pop_Pct'].fillna(-9999)
COVID19_US['Administered_Dose1_Recip_5Plus'] = COVID19_US['Administered_Dose1_Recip_5Plus'].fillna(-9999)
COVID19_US['Administered_Dose1_Recip_5PlusPop_Pct'] = COVID19_US['Administered_Dose1_Recip_5PlusPop_Pct'].fillna(-9999)
COVID19_US['Booster_Doses'] = COVID19_US['Booster_Doses'].fillna(-9999)
COVID19_US['Booster_Doses_Vax_Pct'] = COVID19_US['Booster_Doses_Vax_Pct'].fillna(-9999)
##                              REPLACE null Hospital Bed values with '0'
#These rows are empty cause hospitals in most states didn’t provided the data for certain 
#variable/ columns, Some dates don’t have a rows | Values for covid specific columns didn’t 
#exist at the start of 01/22/2020
COVID19_US['critical_staffing_shortage_today_yes'] = COVID19_US['critical_staffing_shortage_today_yes'].fillna(-9999)
COVID19_US['inpatient_beds'] = COVID19_US['inpatient_beds'].fillna(-9999)
COVID19_US['inpatient_beds_used'] = COVID19_US['inpatient_beds_used'].fillna(-9999)
COVID19_US['inpatient_beds_used_covid'] = COVID19_US['inpatient_beds_used_covid'].fillna(-9999)
COVID19_US['staffed_adult_icu_bed_occupancy'] = COVID19_US['staffed_adult_icu_bed_occupancy'].fillna(-9999)
COVID19_US['staffed_icu_adult_patients_confirmed_and_suspected_covid'] = COVID19_US['staffed_icu_adult_patients_confirmed_and_suspected_covid'].fillna(-9999)
COVID19_US['staffed_icu_adult_patients_confirmed_covid'] = COVID19_US['staffed_icu_adult_patients_confirmed_covid'].fillna(-9999)
COVID19_US['total_adult_patients_hospitalized_confirmed_and_suspected_covid'] = COVID19_US['total_adult_patients_hospitalized_confirmed_and_suspected_covid'].fillna(-9999)
COVID19_US['inpatient_beds_utilization'] = COVID19_US['inpatient_beds_utilization'].fillna(-9999)
COVID19_US['percent_of_inpatients_with_covid'] = COVID19_US['percent_of_inpatients_with_covid'].fillna(-9999)
COVID19_US['inpatient_bed_covid_utilization'] = COVID19_US['inpatient_bed_covid_utilization'].fillna(-9999)
#-----newish----
COVID19_US['inpatient_beds_utilization_numerator'] = COVID19_US['inpatient_beds_utilization_numerator'].fillna(-9999)
COVID19_US['inpatient_beds_utilization_denominator'] = COVID19_US['inpatient_beds_utilization_denominator'].fillna(-9999)
COVID19_US['percent_of_inpatients_with_covid_numerator'] = COVID19_US['percent_of_inpatients_with_covid_numerator'].fillna(-9999)
COVID19_US['percent_of_inpatients_with_covid_denominator'] = COVID19_US['percent_of_inpatients_with_covid_denominator'].fillna(-9999)
COVID19_US['inpatient_bed_covid_utilization_numerator'] = COVID19_US['inpatient_bed_covid_utilization_numerator'].fillna(-9999)
COVID19_US['inpatient_bed_covid_utilization_denominator'] = COVID19_US['inpatient_bed_covid_utilization_denominator'].fillna(-9999)
COVID19_US['adult_icu_bed_covid_utilization'] = COVID19_US['adult_icu_bed_covid_utilization'].fillna(-9999)
COVID19_US['adult_icu_bed_covid_utilization_numerator'] = COVID19_US['adult_icu_bed_covid_utilization_numerator'].fillna(-9999)
COVID19_US['adult_icu_bed_covid_utilization_denominator'] = COVID19_US['adult_icu_bed_covid_utilization_denominator'].fillna(-9999)
COVID19_US['adult_icu_bed_utilization'] = COVID19_US['adult_icu_bed_utilization'].fillna(-9999)
COVID19_US['adult_icu_bed_utilization_numerator'] = COVID19_US['adult_icu_bed_utilization_numerator'].fillna(-9999)
COVID19_US['adult_icu_bed_utilization_denominator'] = COVID19_US['adult_icu_bed_utilization_denominator'].fillna(-9999)







#Nulls were removed at this stage and in the manner in hopes of minimising data waste.
#Shape Before = 
#Shape After = 
#                     Drop Row
#DROP 7140 rows from FIPS column where [value=null] 
#DROP 41412 rows from Admin2 column where [value=null]
#DROP 39984 rows from Metro_Status column where [value=null]
#DROP 55692 rows, where SVI columns are null(This is also all occurances of [State = Puerto Rico] )
#Detailed info on why is in "Data Literature" page in apple note app

# IDEA FROM= 
#Drop Row= https://www.geeksforgeeks.org/drop-rows-from-pandas-dataframe-with-missing-values-or-nan-in-columns/
#REPLACE null= https://www.geeksforgeeks.org/replace-all-the-nan-values-with-zeros-in-a-column-of-a-pandas-dataframe/


##                   WANTED TO DO but DIDNT
## (Below code: when the "county" value for a row is 
## null the fill it up with the value from "state" column 
## for that row. This ensures data not attributed to a 
## county get attributed to the wider state.)
##------
# confirmed_cases_US['Admin2'].fillna(confirmed_cases_US['Province_State'], inplace=True)
# deaths_US['Admin2'].fillna(deaths_US['Province_State'], inplace=True)
# vaccination_US['Recip_County'].fillna(vaccination_US['Recip_State'], inplace=True)


#### Rename Columns

In [18]:
## Rename Columns
COVID19_US.rename(columns={'Admin2': 'County', 'Province_State': 'State', 'State': 'State_Abr'}, inplace=True)

#                     Renaming
# IDEA FROM= 
# https://www.geeksforgeeks.org/how-to-rename-columns-in-pandas-dataframe/

#### Reindexing Columns

In [19]:
## Reindexing Columns (Rearrange the order of columns)
Sorted_column = ['Date', 'State_Abr', 'State', 'County', 'FIPS', 'Metro_Status',
                    'Population', 'Confirmed_Cases', 'Deaths', 'Lat', 'Long_', 'AREA_SQMI', 'E_HU',
                    'E_HH', 'E_POV', 'E_UNEMP', 'E_PCI', 'E_AGE65',
                    'E_AGE17', 'E_MINRTY', 'E_UNINSUR', 'EP_POV', 'EP_UNEMP',
                    'EP_PCI', 'EP_AGE65', 'EP_AGE17', 'EP_MINRTY', 'EP_UNINSUR',
                    'Series_Complete_Pop_Pct', 'Series_Complete_Yes', 'Completeness_pct', 'Administered_Dose1_Recip', 'Administered_Dose1_Pop_Pct',
                    'Administered_Dose1_Recip_5Plus', 'Administered_Dose1_Recip_5PlusPop_Pct', 'Booster_Doses', 'Booster_Doses_Vax_Pct',
                 'inpatient_beds', 'inpatient_beds_used',
                 'inpatient_beds_used_covid', 'critical_staffing_shortage_today_yes', 
                 'inpatient_beds_utilization', 'inpatient_beds_utilization_numerator', 
                 'inpatient_beds_utilization_denominator', 'percent_of_inpatients_with_covid', 
                 'percent_of_inpatients_with_covid_numerator', 'percent_of_inpatients_with_covid_denominator', 
                 'inpatient_bed_covid_utilization', 'inpatient_bed_covid_utilization_numerator', 
                 'inpatient_bed_covid_utilization_denominator', 'staffed_adult_icu_bed_occupancy', 
                 'staffed_icu_adult_patients_confirmed_and_suspected_covid', 'staffed_icu_adult_patients_confirmed_covid', 
                 'total_adult_patients_hospitalized_confirmed_and_suspected_covid', 'adult_icu_bed_covid_utilization', 
                 'adult_icu_bed_covid_utilization_numerator', 'adult_icu_bed_covid_utilization_denominator', 
                 'adult_icu_bed_utilization', 'adult_icu_bed_utilization_numerator', 
                 'adult_icu_bed_utilization_denominator']

COVID19_US = COVID19_US.reindex(Sorted_column, axis='columns')

 
#                     What And Why:
# IDEA FROM= 
# https://www.geeksforgeeks.org/reindexing-in-pandas-dataframe/
# https://www.kite.com/python/answers/how-to-reorder-columns-in-a-pandas-dataframe-in-python

#### Export CSV File

In [20]:

# COVID19_US.to_csv("/Users/emmanueljumbo/dataScience/python/Dissertation/experimentation/data/Ultimate_Covid19_DataSet.csv")




#                     What And Why:
# New dataset will be used for Covid data visualisation, exploration and creation of machine learning models in other “.ipynb” files. 
# IDEA FROM=
# https://www.geeksforgeeks.org/saving-a-pandas-dataframe-as-a-csv/
